# Montioring a RAG chain with OpenAI, LangChain, and MLflow

This notebook is a quick tutorial on how to use [MLflow Tracing](https://mlflow.org/docs/latest/llms/tracing/index.html) to improve observability in your Retrieval Augmented Generation (RAG) application. RAG chains can be complex with many steps involved, and when failures or unexpected responses happen, it can be difficult to pinpoint what exactly went wrong.

MLflow Tracing helps by allowing you to view the inputs and outputs of each intermediate step in your workflow, which enables more effective debugging and iteration. More than that, MLflow has an [integration with Jupyter Notebooks](https://mlflow.org/docs/latest/llms/tracing/index.html#jupyter-notebook-integration) that allows the trace UI to show up inside notebook output cells, allowing you to browse the trace without having to context switch.

To demonstrate these functionalities, we'll be building a simple question answering app. For convenience, we're using [LangChain](https://www.langchain.com/) here (as MLflow has a built-in integration with it), but traces can be instrumented manually to suit any use-case. Let's dive in!

## Setting up the environment

The following cells simply set up our dev environment, installing the necessary libraries, and setting our OpenAI key. 

In [ ]:
%pip install -Uqq mlflow langchain langchain-openai

In [1]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

Enter your OpenAI API Key:  ········


## Setting up the MLflow Tracking Server (optional but recommended)

The [MLflow Tracking Server](https://mlflow.org/docs/latest/tracking/server.html) is a simple HTTP server that provides a web UI and several REST endpoints for tracking runs and experiments. For the purposes of this tutorial, it enables us to view a visualization of our trace, making it easy to browse through the data.

You can run the cell below to start the server, but it might be more convenient (for terminating / cleaning up the process) to run `mlflow server` in an external shell in the directory that contains this notebook.

In [ ]:
from subprocess import Popen

p = Popen(["mlflow", "server"])

In order to terminate the server, you can simply restart the kernel that the notebook is attached to.

## Enabling tracing

When using LangChain, tracing is enabled automatically by simply calling `mlflow.langchain.autolog()`. Manual trace instrumentation is also possible via the `@mlflow.trace()` function decorator. For more details, please check out the [MLflow Docs](https://mlflow.org/docs/latest/llms/tracing/index.html#trace-decorator).

In [2]:
import mlflow

In [ ]:
# skip this cell if not running the tracking server
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
mlflow.set_experiment("openai-rag-demo")
mlflow.langchain.autolog()

2025/02/04 14:48:19 INFO mlflow.tracking.fluent: Experiment with name 'openai-rag-demo' does not exist. Creating a new experiment.


## Setting up our vector store

The first step when building a RAG app is embedding our documents. Here, we'll be using OpenAI's `text-embedding-3-small` model to generate the embeddings, and storing them in an in-memory vector store. We'll be querying this vector store later to fetch documents that are relevant to the user's query.

In [4]:
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

documents = [
    Document(
        page_content="NumPy is a powerful Python library used for numerical computing. It provides support for large multidimensional arrays and matrices along with a collection of high-level mathematical functions."
    ),
    Document(
        page_content="Pandas is a Python library primarily used for data manipulation and analysis. It provides data structures like DataFrame, which makes working with structured data intuitive and efficient."
    ),
    Document(
        page_content="PyTorch is an open-source machine learning library. It is widely used for deep learning applications and provides dynamic computational graphs for flexibility."
    ),
    Document(
        page_content="MLflow is an open-source platform to manage the machine learning lifecycle. It supports tracking experiments, packaging code into reproducible runs, and sharing and deploying models."
    ),
    Document(
        page_content="Langchain is an open-source library designed to build applications powered by language models. It provides a flexible interface to chain together components like prompts, memory, and tools."
    ),
]

# Provide the `persist_directory` argument to save the vector store to disk
vectorstore = InMemoryVectorStore.from_documents(
    documents,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
)

## Building our chain

Our full RAG chain has the following steps:

1. Embed the question and fetch relevant documents
2. Construct a prompt using the fetched documents
3. Feed the prompt to the chat model (in this case, we'll be using OpenAI's `gpt-4o-mini`)

The following cell sets each of these steps up, and links them together to produce the final chain.

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# This handles Step #1. The retriever will be invoked using the user's question,
# and will perform a similarity search over the embedded documents to retrieve
# the top 3 most relevant ones.
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

# This handles Step #2. Placeholders are used to inject
# both the user's question and the retrieved context.
prompt = ChatPromptTemplate.from_template("""
Answer the following question using the provided context. If the information
required to answer the question is not contained within the context, simply
respond that you do not know.

{question}

Context:
{context}
""")

# This handles Step #3. Feel free to change the
# parameters of the modelto whatever you wish!
chat_model = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0,
  max_tokens=50,
)

# Finally, we link them all together using LangChain Expression Language's pipe operator 
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | chat_model

Once the chain is executed (i.e. when you run the cell below), MLflow automatically traces the execution of each of the chain's constituent parts. If you're using a tracking server, you should see a UI appear in the cell output.

This visualization allows you to easily step through the various intermediate steps in the chain's execution, and to see the inputs and outputs at a glance. The UI also supports searching through spans to make it easy to debug unexpected responses even in very large and complex applications.

![Trace UI in Jupyter Notebook](../../images/mlflow_trace_jupyter.png)

In [6]:
# Now, we can simply invoke the chain with our question,
# and the full pipeline will be executed!
chain.invoke("What is MLflow?")

AIMessage(content='MLflow is an open-source platform to manage the machine learning lifecycle. It supports tracking experiments, packaging code into reproducible runs, and sharing and deploying models.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 240, 'total_tokens': 273, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-330c09ea-413a-43cc-a6e3-c1eb6e73abdf-0', usage_metadata={'input_tokens': 240, 'output_tokens': 33, 'total_tokens': 273, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Viewing the trace in the MLflow Web UI

Alternatively, if you'd like to browse historical traces, you can do so via the MLflow Web UI (located at the URI where your tracking server is running--by default, this should be http://127.0.0.1:5000).

In this tutorial, we've created the "openai-rag-demo" experiment, and this is where the trace data is stored. Navigate to the "Traces" tab within this experiment:

![MLflow Web UI](../../images/mlflow_ui.png)

Clicking on the links in the "Request ID" column will open the trace visualization, which is identical to the one that appears in the notebook:

![MLflow Trace UI](../../images/mlflow_trace_ui.png)

Of course, the trace data is accessible via the python client as well. Simply call `mlflow.get_last_active_trace()`, and call `.to_dict()` on the result to convert it to a python dictionary. The `mlflow.search_traces()` and `mlflow.get_trace()` [APIs](https://mlflow.org/docs/latest/llms/tracing/index.html#searching-and-retrieving-traces) are also available to retrieve traces given certain filter conditions.

In [7]:
mlflow.get_last_active_trace().to_dict()

{'info': {'request_id': '89f68d8ff9e245a58708294482b154b8',
  'experiment_id': '423469103627487276',
  'timestamp_ms': 1738651707198,
  'execution_time_ms': 2662,
  'status': 'OK',
  'request_metadata': {'mlflow.trace_schema.version': '2',
   'mlflow.traceInputs': '"What is MLflow?"',
   'mlflow.traceOutputs': '{"content": "MLflow is an open-source platform to manage the machine learning lifecycle. It supports tracking experiments, packaging code into reproducible runs, and sharing and deploying models.", "additional_kwargs": {"refusal": null}, "response...'},
  'tags': {'mlflow.source.name': '/Users/daniel.lok/miniconda3/envs/dev/lib/python3.9/site-packages/ipykernel_launcher.py',
   'mlflow.source.type': 'LOCAL',
   'mlflow.traceName': 'RunnableSequence',
   'mlflow.artifactLocation': 'file:///Users/daniel.lok/openai-cookbook/examples/third_party/mlruns/423469103627487276/traces/89f68d8ff9e245a58708294482b154b8/artifacts'}},
 'data': {'spans': [{'name': 'RunnableSequence',
    'conte